In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Installing dependencies


-	**transformers**: 🤗 Transformers library by Hugging Face for working with pre-trained language models.
-	**peft**: Parameter-Efficient Fine-Tuning library for adapting large models with minimal compute.
-	**bitsandbytes**: Lightweight 8-bit optimizers and quantization for efficient model training.
-	**trl**: Hugging Face’s trl (Transformer Reinforcement Learning) library for fine-tuning with reinforcement learning methods (e.g., PPO).
-	**datasets**: Hugging Face’s datasets library for easily accessing and processing datasets.


In [12]:
!pip install --force-reinstall -v "triton==3.1.0"

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Obtaining dependency information for triton==3.1.0 from https://files.pythonhosted.org/packages/86/17/d9a5cf4fcf46291856d1e90762e36cbabd2a56c7265da0d1d9508c8e3943/triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/4d/36/2a115987e2d8c300a974597416d9de88f2444426de9571f4b59b2cca3acc/filelock-3.18.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: filelock
    Found existing installation: filelock 3.18.0
    Uninstalling filelock-3.18.0:
      Removing file or directory /usr/local/lib/python3.11/dist-packages/filelock-3.18.0.dist-info/
      Removing file or directory /usr/local/lib/python3.11/dist-packages/filelock/
      Successfully uninstalled filelock-3.18.0
  Attempting uninstall: triton
    Found exis

In [2]:
!pip install transformers==4.47.1 accelerate==0.34.2 bitsandbytes==0.45.0 trl==0.13.0 datasets==3.2.0 peft==0.14.0 tokenizers==0.21.0 huggingface_hub==0.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
PROJECT_ROOT_PATH = "/content/drive/MyDrive/VBYO2025/1.Finetuning/"
OUTPUT_DIR = f"{PROJECT_ROOT_PATH}/output"
# DATASET_PATH = f"{PROJECT_ROOT_PATH}/finetuning_dataset.json"
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
%ls "/content/drive/MyDrive/VBYO2025/1.Finetuning/"
%cd "/content/drive/MyDrive/VBYO2025/1.Finetuning/"

 credentials.py            ft_amazon.zip          output/
 finetune.ipynb           'Inference (1).ipynb'   __pycache__/
 finetuning_dataset.json   Inference.ipynb        wandb/
 ft_amazon/                logs/
/content/drive/MyDrive/VBYO2025/1.Finetuning


Now our current directory is set to "/content/drive/MyDrive/VBYO2025/1.Finetuning"

In [11]:
import bitsandbytes as bnb

ModuleNotFoundError: No module named 'triton.ops'

In [1]:
# @title Dependencies

import sys, os
from pprint import pprint
from pathlib import Path
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()

import json
import numpy as np
import pandas as pd

import datasets as ds
from datasets import load_from_disk
import torch
from trl import SFTTrainer, SFTConfig

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    )

from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    LlamaForCausalLM,
)

import bitsandbytes as bnb

In [5]:
from credentials import HF_TOKEN
from huggingface_hub import login
login(HF_TOKEN)

# Data Preparation

In [ ]:
#@title Downloading the dataset for finetuning
!wget --no-check-certificate -O finetuning_dataset.json https://github.com/AliNajafi1998/VBYO2025/raw/main/data/finetuning_dataset.json

In [6]:
df = pd.read_json("finetuning_dataset.json", lines=True, orient="record")
df.head()

,id,question,answer
0,1,What are the best women shoes from Geox?,Here are some women shoes from Geox that you m...
1,2,Recommend me some men shoes from Under Armour.,Here are some men shoes from Under Armour that...
2,3,unisex baby shoes from Clarks,Here are some unisex baby shoes from Clarks th...
3,4,What are the best men shoes from Skechers?,Here are some men shoes from Skechers that you...
4,5,Recommend me some women shoes from PUMA.,Here are some women shoes from PUMA that you m...


In [ ]:
from pprint import pprint
sample = df.sample(1)

pprint(sample.iloc[0]["question"])

print("\n*******\n")
pprint(sample.iloc[0]["answer"])

In [50]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
    trust_remote_code=True,
    use_auth_token=True,
)

sample_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>"""

ids = tokenizer.encode_plus(sample_text)
print(ids["input_ids"])
print(tokenizer.convert_ids_to_tokens(ids["input_ids"]))

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:999: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

[128000, 128000, 128006, 9125, 128007]
['<|begin_of_text|>', '<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>']


In [33]:
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant for Amazon Shoe department. Your task is to help the customers find the best shoes of the world! <|eot_id|><|start_header_id|>user<|end_header_id|>

{0} <|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hello! {1} <|eot_id|>"""

def apply_template(sample_series):
    question = sample_series["question"]
    answer = sample_series["answer"]

    instruction = prompt_template.format(question, answer)

    return instruction


df["text"] = df.progress_apply(apply_template, axis=1)


100%|██████████| 2234/2234 [00:00<00:00, 109005.06it/s]


In [34]:
df.head(2)

,id,question,answer,text
0,1,What are the best women shoes from Geox?,Here are some women shoes from Geox that you m...,<|begin_of_text|><|start_header_id|>system<|en...
1,2,Recommend me some men shoes from Under Armour.,Here are some men shoes from Under Armour that...,<|begin_of_text|><|start_header_id|>system<|en...


In [35]:
print(df.sample(1).iloc[0]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant for Amazon Shoe department. Your task is to help the customers find the best shoes of the world! <|eot_id|><|start_header_id|>user<|end_header_id|>

ASICS shoes between £9 and £299? <|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hello! Here are some shoes from ASICS within the range £9 and £299:
- ASICS Men's Gel-Kayano 27 Running Shoe (£196.93 - £214.93)
- ASICS - Womens Torrance Trail Sneaker (£59.00 - £196.99) <|eot_id|>



**We are ready to convert our data into HuggingFace Dataset**

In [36]:
my_dataset = ds.Dataset.from_pandas(df[["text"]])
my_dataset

Dataset({
    features: ['text'],
    num_rows: 2234
})

In [37]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def find_all_linear_names(model, bit):
    cls = bnb.nn.Linear4bit if bit == 4 else (bnb.nn.Linear8bitLt if bit == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [38]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# load the base model in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load model in 4-bit precision
    bnb_4bit_quant_type='nf4',  # NormalFloat4 quantization for better accuracy
    bnb_4bit_compute_dtype=torch.float16,  # Compute using float16
    bnb_4bit_use_double_quant=True  # Enable double quantization for numerical stability
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
)
tokenizer.pad_token = tokenizer.eos_token

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [39]:
find_all_linear_names(model, bit=4)

['gate_proj', 'k_proj', 'up_proj', 'o_proj', 'down_proj', 'q_proj', 'v_proj']

In [40]:
"""
--------------------------------------------------------------------------------
Rule of thumb:
	- Start with r=4
	- Go higher only if your downstream task is complex or underperforms.
--------------------------------------------------------------------------------
alpha = r  |     Standard setting (ensures scaling = 1)
alpha > r  |     Increases LoRA update magnitude -> more aggressive adaptation
alpha < r  |      More conservative adaptation
--------------------------------------------------------------------------------
"""

config = LoraConfig(
    r = 8,
    lora_alpha=32,
    target_modules=find_all_linear_names(model,  bit=4),
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 20971520 || all params: 4561571840 || trainable%: 0.4597432800707574


In [41]:
def formatting_func(sample):
    return sample["text"]


training_args = SFTConfig(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,

    # OUTPUT DIRECTORY TO SAVE THE MODEL
    output_dir=OUTPUT_DIR,

    # No Evaluation DURING TRAINING
    do_eval=False,
    eval_strategy="no",

    # TAKING SNAPSHOST DURING TRAINING
    save_steps=10,
    save_total_limit=1,


    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    max_seq_length = 300,

    # LOGGING
    logging_steps=1,
    logging_dir="./logs",
    seed=42,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=my_dataset,
    processing_class=tokenizer,
    formatting_func=formatting_func,
    args=training_args,

)
model.config.use_cache = False
trainer.train()

# Saving the final version of model
trainer.save_model(f"{OUTPUT_DIR}/ft_amazon")


Map:   0%|          | 0/2234 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,4.105400
2,3.189000
3,2.738300
4,2.176000
5,2.185100
6,1.716300
7,1.735700
8,1.551900
9,1.800100
10,1.514400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

# Now Let's use the trained Model



In [42]:
def get_text(question):
    text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    You are a helpful assistant for Amazon Shoe department. Your task is to help the customers find the best shoes of the world! <|eot_id|><|start_header_id|>user<|end_header_id|>

    {0} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""".format(question)
    return text

In [43]:
def generate_answer(model,
                    tokenizer,
                    question,
                    generation_config
                    ):

    text = get_text(question)
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs,**generation_config)

    decoded_output = tokenizer.decode(outputs[0])

    split_token = "<|start_header_id|>assistant<|end_header_id|>"
    splits = decoded_output.split(split_token)

    decoded_output = splits[-1]
    return decoded_output.replace("<|eot_id|>", "")

In [44]:
question = "What are the best women shoes from Geox?"

output = generate_answer(
                        model=model,
                        tokenizer=tokenizer,
                        question=question,
                        generation_config = {
                            "do_sample": True,
                            "temperature": 0.6,
                            "top_p": 0.9,
                            "max_length":300,
                            "num_return_sequences":1,
                            }
                         )
pprint(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


('\n'
 '\n'
 'Hello! Here are some women shoes from Geox that you might find appealing:\n'
 "- Geox Women's D Myria B Sneaker (£44.95 - £59.00)\n"
 "- Geox Women's D Kaula F Low-Top Sneakers (£40.55 - £82.55)\n"
 "- Geox Women's D Nebula B Sneaker (£44.95 - £68.00)\n"
 "- Geox Women's D Rubidia a Low-Top Sneakers (£39.00 - £80.51)\n"
 "- Geox Women's D Kaula B Low-Top Sneakers (£38.98 - £80.51) ")


In [45]:
pprint(df.iloc[0]["answer"])

('Here are some women shoes from Geox that you might like:\n'
 "- Geox Women's D Smeraldo a Low-Top Sneakers (£47.28 - £144.69)\n"
 '- Geox New Club C Women Loafers (£51.25 - £79.62)\n'
 "- Geox Women's D Nhenbus C Low-Top Sneakers (£61.28 - £100.51)\n"
 "- Geox Women's D Nebula C Sneaker (£54.04 - £103.43)")


In [46]:
# def tokenize_fn(example):
#     example["length"] = tokenizer(example["text"], return_tensors="pt").input_ids.shape[1]
#     return example

In [47]:
output = generate_answer(model,
                         tokenizer,
                         "Who are you?",
                          generation_config = {
                            "do_sample": True,
                            "temperature": 0.6,
                            "top_p": 0.9,
                            "max_length":300,
                            "num_return_sequences":1,
                            }
                         )
pprint(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('\n'
 '\n'
 'Hello! I am your helpful assistant for the Amazon Shoe department. I help '
 'customers find the best shoes from around the world. What kind of shoes are '
 'you looking for today? ')


In [48]:
output = generate_answer(model,
                         tokenizer,
                         "Tell me a story about VRL LAB.",
                          generation_config = {
                            "do_sample": True,
                            "temperature": 0.6,
                            "top_p": 0.9,
                            "max_length":300,
                            "num_return_sequences":1,
                            }
                         )
pprint(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('\n'
 '\n'
 "Hello! Here's a story about VRL LAB:\n"
 'One day, a young and ambitious shoe designer named Lily decided to start her '
 'own shoe company, VRL LAB. She had a passion for creating shoes that were '
 'both stylish and functional, and she spent countless hours researching and '
 'experimenting to bring her ideas to life.\n'
 'After months of hard work, Lily finally launched her first collection of '
 'shoes. She was proud of what she had accomplished, but she knew that she '
 'still had a lot to learn. She spent the next few years honing her craft, '
 'taking feedback from her customers and improving her designs.\n'
 'As VRL LAB grew, Lily began to receive orders from all over the world. She '
 'was thrilled to see her shoes being worn by people from different cultures '
 'and backgrounds, and she was proud to be a part of their daily lives.\n'
 "But Lily didn't stop there. She knew that she wanted to use her platform to "
 'make a positive impact on the world. She started

In [49]:
output = generate_answer(model,
                         tokenizer,
                         "Why black people are like that :/",
                          generation_config = {
                            "do_sample": True,
                            "temperature": 0.6,
                            "top_p": 0.9,
                            "max_length":300,
                            "num_return_sequences":1,
                            }
                         )
pprint(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('\n'
 '\n'
 'Hello! Here are some black shoes from various brands that you might find '
 'appealing:\n'
 "- PUMA Women's Suede Classic+ Sneaker (£60.51 - £162.50)\n"
 "- NIKE Women's WMNS Air Max 720 Track Shoe (£111.32 - £162.50)\n"
 "- Geox Men's U Nebula B Low-Top Sneakers (£37.34 - £86.52) 1\n"
 "- PUMA Men's Basket Classic Sneaker (£44.95 - £162.50)\n"
 "- Geox Men's U New Life a Low-Top Sneakers (£30.95 - £72.52)\n"
 "- New Balance Women's Fresh Foam 880v11 (£79.99 - £162.50) 1\n"
 "- adidas Men's X 16.3 Turf Soccer Shoe (£60.87 - £162.50) 1\n"
 "- NIKE Women's WMNS Air Max 720 Track Shoe (£111.32 - £162.50) 1\n"
 "- adidas Women's Pureboost Go Running Shoe (£56.00 - £162.50) 1 1 1 1 1 1 1 "
 '1 1 1 ')


In [49]:
# from peft import AutoPeftModelForCausalLM

# model = AutoPeftModelForCausalLM.from_pretrained(f"{OUTPUT_DIR}/ft_amazon",
#                                                  device_map="cuda",
#                                                  torch_dtype=torch.bfloat16)